Notebook for second problem

In [28]:
ts=zip(range(1,10),range(11,20))

In [29]:
for t in ts:
    print(t)
else:
    print("There was a premature break") #This is intereintg

(1, 11)
(2, 12)
(3, 13)
(4, 14)
(5, 15)
(6, 16)
(7, 17)
(8, 18)
(9, 19)
There was a premature break
